# python-can CAN over serial

[```python-can```](https://python-can.readthedocs.io/en/2.1.0/index.html) includes a [CAN over Serial](
https://python-can.readthedocs.io/en/2.1.0/interfaces/serial.html
) module.

This allows prototyping and testing with inexpensive Arduino devices before switching to another module with minimal code changes.

The packet over serial



|                | Length (Byte) | Data type               | Byte order    | Description                                   |
|----------------|---------------|-------------------------|---------------|-----------------------------------------------|
| Start of frame | 1             | Byte                    | -             | Must be 0xAA                                  |
| Timestamp      | 4             | Unsigned 4 byte integer | Little-Endian | Usually s, ms or µs since start of the device |
| DLC            | 1             | Unsigned 1 byte integer | Little-Endian | Length in byte of the payload                 |
| Arbitration ID | 4             | Unsigned 4 byte integer | Little-Endian | -                                             |
| Payload        | 0 - 8         | Byte                    | -             | -                                             |
| End of frame   | 1             | Byte                    | -             | Must be 0xBB                                  |

In [356]:
import glob
import uuid
import jinja2
import os
import re

In [357]:
old_sketches = glob.glob("*.ino")
old_sketches

['06d8aed8-877c-4bdc-83c0-9284998f05b2.ino']

In [358]:
for old_sketch in old_sketches:
    print("Deleting: {}".format(old_sketch))
    os.unlink(old_sketch)

Deleting: 06d8aed8-877c-4bdc-83c0-9284998f05b2.ino


## Universally unique identifier

> From Wikipedia, the free encyclopedia
A universally unique identifier (UUID) is a 128-bit number used to identify information in computer systems. The term globally unique identifier (GUID) is also used.

> When generated according to the standard methods, UUIDs are for practical purposes unique, without depending for their uniqueness on a central registration authority or coordination between the parties generating them, unlike most other numbering schemes. While the probability that a UUID will be duplicated is not zero, it is close enough to zero to be negligible.

> Thus, anyone can create a UUID and use it to identify something with near certainty that the identifier does not duplicate one that has already been, or will be, created to identify something else. **Information labeled with UUIDs by independent parties can therefore be later combined into a single database, or transmitted on the same channel, without needing to resolve conflicts between identifiers**.

> Adoption of UUIDs and GUIDs is widespread, with many computing platforms providing support for generating them, and for parsing their textual representation.

- https://en.wikipedia.org/wiki/Universally_unique_identifier

In [359]:
build_uuid = str(uuid.uuid4())
build_uuid

'4ef725f2-0beb-4755-b3ab-5af50dc810f2'

In [369]:
arduino_template_str = """
unsigned long BAUD = {{ baud }};
unsigned long DELAY = {{ delay }};
char BUILD_UUID[] = "{{ build_uuid }}";

unsigned long arbitration_id = 0x0;
unsigned long time_millis = 0b0;
unsigned long timestamp = 0x0;

uint8_t counter8 = 0x00;
uint16_t counter16 = 0x0000;
uint32_t counter32 = 0x00000000;

// Arduino Setup
void setup() {
Serial.begin(BAUD);
Serial.print("Build UUID: ");
Serial.println(BUILD_UUID);

// Start of frame
Serial.write(0xAA);

// Timestamp
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);

// DLC
Serial.write(0x00);

// Arbitration ID
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);

// End of frame
Serial.write(0xBB);
return; 
}

// Arduino Main Loop
void loop() {
time_millis = millis();
// Start of frame
Serial.write(0xAA);

// Timestamp
timestamp = time_millis/10;
for(char shift=0;shift<32;shift+=8) {
Serial.write((char)(((unsigned long)timestamp>>shift)&0b11111111));
}

// DLC
Serial.write(0x04);

// Arbitration ID
arbitration_id=1;
for(char shift=0;shift<32;shift+=8) {
Serial.write((char)(((unsigned long)arbitration_id>>shift)&0xFF));
}

// Payload
for(char shift=0;shift<32;shift+=8) {
Serial.write((unsigned long)time_millis>>shift&0b11111111);
}
Serial.write(0xBB);
}
"""
arduino_template = jinja2.Template(arduino_template_str)

In [370]:
sketch_cfg=dict()
sketch_cfg["baud"] = 115200
sketch_cfg["delay"]= 1000
sketch_cfg["arbitration_id"]=12
sketch_cfg["build_uuid"]=build_uuid

arduino_sketch = (arduino_template.render(**sketch_cfg))
print(arduino_sketch)

sketch_file = "{}.ino".format(build_uuid)
with open(sketch_file, "w") as fid:
    fid.write(arduino_sketch)


unsigned long BAUD = 115200;
unsigned long DELAY = 1000;
char BUILD_UUID[] = "4ef725f2-0beb-4755-b3ab-5af50dc810f2";

unsigned long arbitration_id = 0x0;
unsigned long time_millis = 0b0;
unsigned long timestamp = 0x0;

uint8_t counter8 = 0x00;
uint16_t counter16 = 0x0000;
uint32_t counter32 = 0x00000000;

// Arduino Setup
void setup() {
Serial.begin(BAUD);
Serial.print("Build UUID: ");
Serial.println(BUILD_UUID);

// Start of frame
Serial.write(0xAA);

// Timestamp
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);

// DLC
Serial.write(0x00);

// Arbitration ID
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);
Serial.write(0x00);

// End of frame
Serial.write(0xBB);
return; 
}

// Arduino Main Loop
void loop() {
time_millis = millis();
// Start of frame
Serial.write(0xAA);

// Timestamp
timestamp = time_millis/10;
for(char shift=0;shift<32;shift+=8) {
Serial.write((char)(((unsigned long)timestamp>>shift)&0b11111111));
}

// DLC
Serial.write(0x04)

In [371]:
import subprocess
subprocess.check_output(["clang-format-6.0", "-style=LLVM", "-i", sketch_file])

b''

In [372]:
try:
    stdout = subprocess.check_output(["make", "upload"]).decode()
    print(stdout)
except subprocess.CalledProcessError as err:
    print("Build Failed:")
    print(err)
except:
    raise

-------------------------
Arduino.mk Configuration:
- [AUTODETECTED]       CURRENT_OS = LINUX 
- [AUTODETECTED]       ARDUINO_DIR = /usr/share/arduino 
- [COMPUTED]           ARDMK_DIR = /projects/arduino_IO/arduino_make (relative to Common.mk)
- [AUTODETECTED]       ARDUINO_VERSION = 105 
- [DEFAULT]            ARCHITECTURE =  
- [DEFAULT]            ARDMK_VENDOR = arduino 
- [AUTODETECTED]       ARDUINO_PREFERENCES_PATH = /mnt/ubuntu1604_2/home/jed/.arduino/preferences.txt 
- [AUTODETECTED]       ARDUINO_SKETCHBOOK = /mnt/ubuntu1604_2/home/jed/sketchbook (from arduino preferences file)
- [BUNDLED]            AVR_TOOLS_DIR = /usr/share/arduino/hardware/tools/avr (in Arduino distribution)
- [COMPUTED]           ARDUINO_LIB_PATH = /usr/share/arduino/libraries (from ARDUINO_DIR)
- [COMPUTED]           ARDUINO_VAR_PATH = /usr/share/arduino/hardware/arduino//variants (from ARDUINO_DIR)
- [COMPUTED]           BOARDS_TXT = /usr/share/arduino/hardware/arduino//boards.txt (from ARDUINO_DIR)
- 

In [373]:
serial_ports = re.compile("(/dev/ttyUSB[\d]+)").findall(stdout)
serial_ports

['/dev/ttyUSB2', '/dev/ttyUSB2', '/dev/ttyUSB2']

In [374]:
for i in range(len(serial_ports)-1):
    assert (serial_ports[i] == serial_ports[i+1])

In [375]:
import serial
import time

In [386]:
ser = serial.Serial(port=serial_ports[0], baudrate=115200,
                     bytesize=serial.EIGHTBITS,
                     parity=serial.PARITY_NONE,
                     stopbits=serial.STOPBITS_ONE,
                     timeout=10,
                     xonxoff=0,
                     rtscts=0
                   )

In [388]:
ser.setDTR(False) # Drop DTR
time.sleep(1)   # Read somewhere that 22ms is what the UI does.
ser.flushInput()
ser.flushOutput()
ser.flush()
ser.setDTR(True)  # UP the DTR back
header = ser.readline()
header

b'\x00\x00\x00fV\x00\x00\xbb\xaa\xa3\x08\x00\x00\x04\x01\x00\x00\x00\xe7Build UUID: 4ef725f2-0beb-4755-b3ab-5af50dc810f2\r\n'

In [392]:
header_uuid = header.split(b":")[1].strip().decode("UTF-8")

In [393]:
assert build_uuid == header_uuid

In [396]:
ser.setDTR(False) # Drop DTR
time.sleep(1)   # Read somewhere that 22ms is what the UI does.
ser.flushInput()
ser.flushOutput()
ser.flush()
ser.setDTR(True)  # UP the DTR back

header_packet = ser.read_until(b"\xBB")
header_packet

b'\x01\x00\x00\x00)\x02\x00\x00\xbb'

In [317]:
header_packet.split(b"\xAA")

[b'Build UUID: 06d8aed8-877c-4bdc-83c0-9284998f05b2\r\n',
 b'\x00\x00\x00\x00\x00\x00\x00\x00\xff\xbb']

In [322]:
for x in range(10):
    ser.read_until(b"\xAA")
    packet = ser.read_until(b"\xBB")
    print(packet)

b'\xe8.\x00\x00\x01\x01\x00\x00\x00\x0c\xbb'
b'\xe8.\x00\x00\x02\x02\x00\x00\x00\xbb'
b'\xd02\x00\x00\x04\x00\x00\x00\x00\xd02\x00\x00\xbb'
b'\xd02\x00\x00\x01\x01\x00\x00\x00\r\xbb'
b'\xd02\x00\x00\x02\x02\x00\x00\x00\xbb'
b'\xb96\x00\x00\x04\x00\x00\x00\x00\xb96\x00\x00\xbb'
b'\xb96\x00\x00\x01\x01\x00\x00\x00\x0e\xbb'
b'\xb96\x00\x00\x02\x02\x00\x00\x00\xbb'
b'\xa1:\x00\x00\x04\x00\x00\x00\x00\xa1:\x00\x00\xbb'
b'\xa1:\x00\x00\x01\x01\x00\x00\x00\x0f\xbb'


In [305]:
)packet = ser.read_until(b"\xBB")
packet

b'\xaa\xbb'

In [309]:
packet = ser.read_until(b"\xBB")
packet

b'\xaa\xa3\x0f\x00\x00\x01\x01\x00\x00\x00\x04\xbb'

In [209]:
packet[1:5]

b'\xe9\x03\x00\x00'

In [210]:
timestamp_raw = packet[1:5]
timestamp_raw

b'\xe9\x03\x00\x00'

In [211]:
int.from_bytes(timestamp_raw, 'little')

1001

In [212]:
int.from_bytes(timestamp_raw, 'big')

3909287936

In [213]:
timestamp = int.from_bytes(timestamp_raw, 'little')
timestamp

1001

In [214]:
sof = packet[0]
sof

170

In [215]:
int.from_bytes(b'\xAA', 'little')

170

In [216]:
int.from_bytes(b'\xAA', 'big')

170

In [217]:
assert sof == int.from_bytes(b'\xAA', 'little')

In [218]:
dlc = packet[5]
dlc

4

In [219]:
arbitration_id_raw = packet[6:10]
arbitration_id_raw

b'\x00\x00\x00\x00'

In [220]:
int.from_bytes(arbitration_id_raw, 'little')

0

In [221]:
int.from_bytes(arbitration_id_raw, 'big')

0

In [222]:
data = packet[11]
data 

3

In [223]:
sketch_cfg

{'baud': 115200,
 'delay': 1000,
 'arbitration_id': 12,
 'build_uuid': '06d8aed8-877c-4bdc-83c0-9284998f05b2'}

In [224]:
header_uuid

'06d8aed8-877c-4bdc-83c0-9284998f05b2'

In [225]:
sketch_cfg["build_uuid"]

'06d8aed8-877c-4bdc-83c0-9284998f05b2'

In [226]:
assert str(header_uuid) == header_uuid

In [227]:
assert str(sketch_cfg["build_uuid"]) == sketch_cfg["build_uuid"]

In [241]:
assert header_uuid == sketch_cfg["build_uuid"]

In [242]:
ser.close()

In [243]:
import can
can.__version__

'2.1.0'

In [244]:
bus_cfg = dict()
bus_cfg["bustype"] = "serial"
bus_cfg["channel"]=serial_ports[0]
bus_cfg["bitrate"]=115200

bus = can.interface.Bus(**bus_cfg)    

In [245]:
bus.channel_info

'Serial interface: /dev/ttyUSB0'

In [246]:
packet = bus.recv(timeout=2)
packet

can.Message(timestamp=22.013, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0x0, dlc=4, data=[0xfd, 0x55, 0x0, 0x0])

In [254]:
packet.timestamp

32.019

In [255]:
packet.arbitration_id

0

In [256]:
packet.dlc

4

In [257]:
packet.data

bytearray(b'\x13}\x00\x00')

In [258]:
int.from_bytes(packet.data, 'little')

32019

In [259]:
int.from_bytes(packet.data, 'big')

326959104

In [260]:
import time

In [271]:
can_packets_time = list()
can_packets_sine = list()

while len(can_packets_time)<20 & len(can_packets_sine)<20:
    packet = bus.recv(timeout=1000)
    if packet is None:
        time.sleep(0.5)
        continue
    if (packet.arbitration_id == 0):
        can_packets_time.append(packet)
    elif (packet.arbitration_id == 1):
        can_packets_sine.append(packet)
    else:
        raise Exception(packet.arbitration_id)

In [272]:
sketch_cfg["delay"]

1000

In [273]:
can_packets_time

[]

In [274]:
import numpy as np

In [275]:
data=object()

In [169]:
class Data(object):
    def __init__(self):
        pass
data = Data()